In [17]:
import pandas as pd
import numpy as np
import os
import vaex
from datetime import * 

In [151]:
path = r"F:\myresearch\MIMICIIIsupport\tables"
os.chdir(path)

In [217]:
icustay_orign = pd.read_csv("icustay_detail.csv",parse_dates=['dod', 'admittime','dischtime','intime', 'outtime'])

In [218]:
icustay = pd.read_csv("icustay_detail.csv",parse_dates=['dod', 'admittime','dischtime','intime', 'outtime'])

In [219]:
icustay_orign.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61051 entries, 0 to 61050
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   subject_id            61051 non-null  int64         
 1   hadm_id               61051 non-null  int64         
 2   icustay_id            61051 non-null  int64         
 3   gender                61051 non-null  object        
 4   dod                   23889 non-null  datetime64[ns]
 5   admittime             61051 non-null  datetime64[ns]
 6   dischtime             61051 non-null  datetime64[ns]
 7   los_hospital          61051 non-null  float64       
 8   admission_age         61051 non-null  float64       
 9   ethnicity             61051 non-null  object        
 10  ethnicity_grouped     61051 non-null  object        
 11  hospital_expire_flag  61051 non-null  int64         
 12  hospstay_seq          61051 non-null  int64         
 13  first_hosp_stay 

In [220]:
## 计算并加入生存信息

icustay.loc[:,"survive_days"] = (icustay.dod-icustay.admittime).apply(lambda x:x.days+1)

icustay.loc[:,"survive_30days"] = icustay["survive_days"].apply(lambda x: 0 if (x>30 or np.isnan(x)) else 1)

icustay.loc[:,"survive_90days"] = icustay["survive_days"].apply(lambda x: 0 if (x>90 or np.isnan(x)) else 1)

icustay.loc[:,"survive_180days"] = icustay["survive_days"].apply(lambda x: 0 if (x>180 or np.isnan(x)) else 1)

icustay.loc[:,"survive_365days"] = icustay["survive_days"].apply(lambda x: 0 if (x>365 or np.isnan(x)) else 1)

icustay.loc[:,"not_survive"] =  icustay["survive_days"].apply(lambda x: 0 if (np.isnan(x)) else 1)

In [221]:
icustay[icustay.columns[icustay.columns.str.contains("survive")][1:]].apply(lambda x:x.value_counts())

,survive_30days,survive_90days,survive_180days,survive_365days,not_survive
0,53524,50250,48053,45635,37162
1,7527,10801,12998,15416,23889


In [272]:
firstday = os.listdir(r".\firstday")

## 读取所有firstday表格
first = {}
for i in firstday:
    first[i] = pd.read_csv(".\\firstday\\"+i)

In [223]:
## 加入首日RRT情况
icustay = pd.merge(icustay,first[firstday[5]][["rrt","icustay_id"]],on="icustay_id",how="left")

## 加入首日机械通气情况
icustay = pd.merge(icustay,first[firstday[7]][["vent","icustay_id"]],on="icustay_id",how="left")

In [224]:
print(icustay.subject_id.drop_duplicates().shape)

print(icustay.hadm_id.drop_duplicates().shape)

print(icustay.icustay_id.drop_duplicates().shape)

## icustay_id是唯一

(46428,)
(57328,)
(61051,)


In [225]:
## 提取首次入ICU病例
icustay=icustay[icustay.icustay_seq==1]

In [226]:
print(icustay.subject_id.drop_duplicates().shape)

print(icustay.hadm_id.drop_duplicates().shape)

print(icustay.icustay_id.drop_duplicates().shape)

## icustay_id是唯一

(46428,)
(57328,)
(57328,)


In [227]:
## 加入首日尿量
icustay = pd.merge(icustay,first[firstday[6]][["urineoutput","icustay_id"]],on="icustay_id",how="left")

In [228]:
## 加入首日GCS评分
first_gcs = first[firstday[2]]

first_gcs["gcs"] = first_gcs.gcseyes+first_gcs.gcsmotor+ first_gcs.gcsverbal

icustay = pd.merge(icustay,first[firstday[2]][["icustay_id","gcs","endotrachflag"]],on="icustay_id",how="left")

In [230]:
## 加入首日化验指标
firstlab = first[firstday[4]] 

firstlab.icustay_id.drop_duplicates().shape

firstlab.hadm_id.drop_duplicates().shape

firstlab = firstlab.groupby("hadm_id").first()

firstlab = firstlab.reset_index()

icustay = pd.merge(icustay,firstlab,on="icustay_id",how="left")

In [232]:
## 加入首日生命体征
firstvital = first[firstday[8]].groupby("hadm_id").first()

firstvital = firstvital.reset_index()

icustay = pd.merge(icustay,firstvital,on="icustay_id",how="left")

In [236]:
## 加入体重
firstweight = first[firstday[9]]

icustay = pd.merge(icustay,firstweight[["weight","icustay_id"]],on="icustay_id",how="left")

In [237]:
## 加入身高
firstheight = first[firstday[3]]
icustay = pd.merge(icustay,firstheight[["height","icustay_id"]],on="icustay_id",how="left")

In [309]:
bga = first[firstday[1]]

In [288]:
bg = first[firstday[0]]

In [295]:
bg = bg[bg.specimen == "ART"]

In [297]:
bg.drop(["specimen","intubated","ventilationrate","ventilator"],axis =1,inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [303]:
bg = bg.groupby("icustay_id").first()

In [317]:
bga = bga.groupby("icustay_id").first().reset_index()

In [322]:
bga= bga[bga.specimen_pred == "ART"]   

In [326]:
bga.drop(['subject_id', 'hadm_id', 'charttime','specimen_pred',
       'specimen_prob'],axis=1,inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [328]:
icustay = pd.merge(icustay,bga,on="icustay_id",how="left")

In [343]:
## 加入血管活性药物使用时间，小时和天数
vasopressor_durations = pd.read_csv(r".\duration\vasopressor_durations.csv")

vasopressor_time = vasopressor_durations.groupby("icustay_id").duration_hours.agg("sum").reset_index()

vasopressor_time.loc[:,"vasopressor_days"] = vasopressor_time["duration_hours"]/24

icustay = pd.merge(icustay,vasopressor_time,on="icustay_id",how="left")

In [359]:
## 加入机械通气时间，小时和天数
ventilation_durations = pd.read_csv(r".\duration\ventilation_durations.csv")

ventilation_durations = ventilation_durations[ventilation_durations.icustay_id.notna()]

ventilation_durations = ventilation_durations.groupby("icustay_id").duration_hours.agg("sum").reset_index()

ventilation_durations.columns = ["icustay_id","vent_hours" ]

ventilation_durations.loc[:,"vent_days"] = ventilation_durations.vent_hours/24

icustay = pd.merge(icustay,ventilation_durations,on="icustay_id",how="left")

In [375]:
## 加入RRT时间和次数
crrt_durations = pd.read_csv(r".\duration\crrt_durations.csv")

crrt_durations = crrt_durations[crrt_durations.icustay_id.notna()]

crrt_timesum = crrt_durations.groupby("icustay_id").duration_hours.agg("sum").reset_index()

crrt_timecount = crrt_durations.groupby("icustay_id").duration_hours.agg("count").reset_index()

icustay = pd.merge(icustay,crrt_timecount,on="icustay_id",how="left")

icustay = pd.merge(icustay,crrt_timesum,on="icustay_id",how="left")

In [396]:
## 加入身高及体重

heightweight = pd.read_csv(r".\demographics\heightweight.csv")

icustay = pd.merge(icustay,heightweight[["icustay_id","weight_first","height_first"]],on="icustay_id",how="left")

icustay.loc[:,"BMI"] = icustay.weight_first/(icustay.height/100)**2

In [406]:
## 加入7天肾衰情况
kdi_7 = pd.read_csv(r".\organfailure\kdigo_stages_7day.csv")

icustay =pd.merge(icustay,kdi_7[["icustay_id","aki_stage_7day","aki_7day"]],on="icustay_id",how="left")

In [410]:
## 加入48H肾衰情况
kdi_48 = pd.read_csv(r".\organfailure\kdigo_stages_48hr.csv")

icustay =pd.merge(icustay,kdi_48[["icustay_id","aki_stage_48hr","aki_48hr"]],on="icustay_id",how="left")

In [415]:
## 加入危重评分 三项
sofa = pd.read_csv(r".\scores\sofa.csv")
sapsii = pd.read_csv(r".\scores\sapsii.csv")
apsiii = pd.read_csv(r".\scores\apsiii.csv")

icustay =pd.merge(icustay,sofa[["icustay_id","sofa"]],on="icustay_id",how="left")

icustay =pd.merge(icustay,sapsii[["icustay_id","sapsii"]],on="icustay_id",how="left")
icustay =pd.merge(icustay,apsiii[["icustay_id","apsiii"]],on="icustay_id",how="left")

In [430]:
## 加入合并症，这个表合并症少，不用这个

comorbidities_ahrq_data=pd.read_csv(r".\comorbidities\comorbidities_ahrq_data.csv")

comorbidities_ahrq_data[comorbidities_ahrq_data.columns[2:]].apply(lambda x:x.value_counts())

In [437]:
## 加入合并症，这个多，用这个
comorbidities_data =pd.read_csv(r".\comorbidities\comorbidities_data.csv")

icustay = pd.merge(icustay,comorbidities_data,left_on="hadm_id_x",right_on="hadm_id",how="left")

In [440]:
pd.DataFrame(icustay.columns,columns=["column_name"]).to_excel("column_names.xlsx")

In [441]:
icustay.shape

(57328, 173)

In [442]:
icustay.to_csv("icustay_final.csv",index=None)